In [ ]:
!pip install Sastrawi
!pip install fuzzywuzzy python-Levenshtein

In [ ]:
import pandas as pd

# Load datasets
laptop_df = pd.read_csv('cleaned_dataset.csv')
min_req_df = pd.read_csv('minimum_requirements_game.csv')
rec_req_df = pd.read_csv('recommended_requirements_game.csv')
cpu_bench_df = pd.read_csv('cpu_benchmarks.csv')
gpu_bench_df = pd.read_csv('gpu_benchmarks.csv')

# Lihat struktur kolom dan sampel data
print("=== Laptop Dataset ===")
print(laptop_df.info())
print(laptop_df.head())

print("\n=== Minimum Requirements ===")
print(min_req_df.info())
print(min_req_df.head())

print("\n=== Recommended Requirements ===")
print(rec_req_df.info())
print(rec_req_df.head())

print("\n=== CPU Benchmark ===")
print(cpu_bench_df.info())
print(cpu_bench_df.head())

print("\n=== GPU Benchmark ===")
print(gpu_bench_df.info())
print(gpu_bench_df.head())

In [ ]:
import re

def split_cpu_vendor(cpu_string):
    """
    Pisahkan CPU Intel dan AMD dari string CPU.
    Return tuple: (intel_cpu, amd_cpu)
    """
    if pd.isna(cpu_string):
        return '', ''
    # Pisahkan model (/, |, ,)
    cpu_models = re.split(r'[\/|,]', cpu_string)
    cpu_models = [m.strip() for m in cpu_models if m.strip()]

    intel_cpu = ''
    amd_cpu = ''

    for cpu in cpu_models:
        if re.search(r'Intel|Pentium|Celeron|Core|Xeon', cpu, re.I):
            intel_cpu = cpu if not intel_cpu else intel_cpu + ' / ' + cpu
        elif re.search(r'AMD|Ryzen|FX|Athlon|Phenom|Opteron', cpu, re.I):
            amd_cpu = cpu if not amd_cpu else amd_cpu + ' / ' + cpu
    return intel_cpu, amd_cpu

def split_gpu_vendor(gpu_string):
    """
    Pisahkan GPU NVIDIA, AMD, dan Intel dari string GPU.
    Return tuple: (nvidia_gpu, amd_gpu, intel_gpu)
    """
    if pd.isna(gpu_string):
        return '', '', ''
    gpu_models = re.split(r'[\/|,]', gpu_string)
    gpu_models = [m.strip() for m in gpu_models if m.strip()]

    nvidia_gpu = ''
    amd_gpu = ''
    intel_gpu = ''

    for gpu in gpu_models:
        if re.search(r'NVIDIA|GeForce', gpu, re.I):
            nvidia_gpu = gpu if not nvidia_gpu else nvidia_gpu + ' / ' + gpu
        elif re.search(r'AMD|Radeon', gpu, re.I):
            amd_gpu = gpu if not amd_gpu else amd_gpu + ' / ' + gpu
        elif re.search(r'Intel|Iris|UHD|HD Graphics', gpu, re.I):
            intel_gpu = gpu if not intel_gpu else intel_gpu + ' / ' + gpu
    return nvidia_gpu, amd_gpu, intel_gpu

In [ ]:
# Pisahkan CPU
cpu_split = rec_req_df['CPU'].apply(lambda x: pd.Series(split_cpu_vendor(x)))
rec_req_df['CPU_Intel'] = cpu_split[0]
rec_req_df['CPU_AMD'] = cpu_split[1]

# Pisahkan GPU
gpu_split = rec_req_df['GPU'].apply(lambda x: pd.Series(split_gpu_vendor(x)))
rec_req_df['GPU_NVIDIA'] = gpu_split[0]
rec_req_df['GPU_AMD'] = gpu_split[1]
rec_req_df['GPU_Intel'] = gpu_split[2]

In [ ]:
# Pisahkan CPU
cpu_split = min_req_df['CPU'].apply(lambda x: pd.Series(split_cpu_vendor(x)))
min_req_df['CPU_Intel'] = cpu_split[0]
min_req_df['CPU_AMD'] = cpu_split[1]

# Pisahkan GPU
gpu_split = min_req_df['GPU'].apply(lambda x: pd.Series(split_gpu_vendor(x)))
min_req_df['GPU_NVIDIA'] = gpu_split[0]
min_req_df['GPU_AMD'] = gpu_split[1]
min_req_df['GPU_Intel'] = gpu_split[2]

In [ ]:
# Drop kolom CPU dan GPU dari minimum requirements
min_req_df = min_req_df.drop(columns=['CPU', 'GPU'])

# Drop kolom CPU dan GPU dari recommended requirements
rec_req_df = rec_req_df.drop(columns=['CPU', 'GPU'])

print("Kolom CPU & GPU udah di-drop dari dataset minimum & recommended!")

In [ ]:
print(rec_req_df.head(10))

In [ ]:
print(min_req_df.head(10))

In [ ]:
# Ubah kolom Category jadi list
min_req_df['Category'] = min_req_df['Category'].str.split(' / ')
rec_req_df['Category'] = rec_req_df['Category'].str.split(' / ')

print("Kolom 'Category' udah jadi list di kedua dataset!")
print(min_req_df[['Game', 'Category']].head())
print(rec_req_df[['Game', 'Category']].head())

In [ ]:
from fuzzywuzzy import process
import pandas as pd

# Buat dict lookup dari benchmark
cpu_lookup = dict(zip(cpu_bench_df['CPU Model'], cpu_bench_df['CPU Score']))
gpu_lookup = dict(zip(gpu_bench_df['GPU Model'], gpu_bench_df['GPU Score']))

# Fungsi matching CPU
def match_cpu_score(cpu_name):
    if pd.isna(cpu_name) or cpu_name.strip() == '':
        return 0
    cpu_list = [c.strip() for c in cpu_name.split('/') if c.strip() != '']
    scores = []
    for cpu in cpu_list:
        match = process.extractOne(cpu, cpu_lookup.keys(), score_cutoff=80)
        if match:
            scores.append(cpu_lookup[match[0]])
    return max(scores, default=0)

# Fungsi matching GPU
def match_gpu_score(gpu_name):
    if pd.isna(gpu_name) or gpu_name.strip() == '':
        return 0
    gpu_list = [g.strip() for g in gpu_name.split('/') if g.strip() != '']
    scores = []
    for gpu in gpu_list:
        if 'Integrated' in gpu or gpu.lower() == 'integrated':
            scores.append(300)
        else:
            match = process.extractOne(gpu, gpu_lookup.keys(), score_cutoff=80)
            if match:
                scores.append(gpu_lookup[match[0]])
    return max(scores, default=0)

# Fungsi isi nilai kosong dari kolom lain
def fill_empty_with_other(row, col1, col2):
    if pd.isna(row[col1]) or row[col1].strip() == '':
        return row[col2]
    if pd.isna(row[col2]) or row[col2].strip() == '':
        return row[col1]
    return row[col1]

# Proses Minimum Requirements
print("⚙️ Proses Minimum Requirements...")

# CPU
min_req_df['CPU_AMD'] = min_req_df.apply(lambda row: fill_empty_with_other(row, 'CPU_AMD', 'CPU_Intel'), axis=1)
min_req_df['CPU_Intel'] = min_req_df.apply(lambda row: fill_empty_with_other(row, 'CPU_Intel', 'CPU_AMD'), axis=1)

# GPU - pastikan semua GPU terisi
min_req_df['GPU_AMD'] = min_req_df.apply(lambda row: fill_empty_with_other(row, 'GPU_AMD', 'GPU_NVIDIA'), axis=1)
min_req_df['GPU_NVIDIA'] = min_req_df.apply(lambda row: fill_empty_with_other(row, 'GPU_NVIDIA', 'GPU_AMD'), axis=1)
min_req_df['GPU_Intel'] = min_req_df.apply(lambda row: fill_empty_with_other(row, 'GPU_Intel', 'GPU_NVIDIA'), axis=1)
min_req_df['GPU_Intel'] = min_req_df.apply(lambda row: fill_empty_with_other(row, 'GPU_Intel', 'GPU_AMD'), axis=1)

# Matching skor
min_req_df['CPU_Intel_score'] = min_req_df['CPU_Intel'].apply(match_cpu_score)
min_req_df['CPU_AMD_score'] = min_req_df['CPU_AMD'].apply(match_cpu_score)
min_req_df['GPU_NVIDIA_score'] = min_req_df['GPU_NVIDIA'].apply(match_gpu_score)
min_req_df['GPU_AMD_score'] = min_req_df['GPU_AMD'].apply(match_gpu_score)
min_req_df['GPU_Intel_score'] = min_req_df['GPU_Intel'].apply(match_gpu_score)

# Update GPU_NVIDIA_score & GPU_AMD_score kalau kosong, pakai GPU_Intel_score
min_req_df.loc[min_req_df['GPU_NVIDIA_score'] == 0, 'GPU_NVIDIA_score'] = min_req_df['GPU_Intel_score']
min_req_df.loc[min_req_df['GPU_AMD_score'] == 0, 'GPU_AMD_score'] = min_req_df['GPU_Intel_score']

# Proses Recommended Requirements
print("⚙️ Proses Recommended Requirements...")

# CPU
rec_req_df['CPU_AMD'] = rec_req_df.apply(lambda row: fill_empty_with_other(row, 'CPU_AMD', 'CPU_Intel'), axis=1)
rec_req_df['CPU_Intel'] = rec_req_df.apply(lambda row: fill_empty_with_other(row, 'CPU_Intel', 'CPU_AMD'), axis=1)

# GPU
rec_req_df['GPU_AMD'] = rec_req_df.apply(lambda row: fill_empty_with_other(row, 'GPU_AMD', 'GPU_NVIDIA'), axis=1)
rec_req_df['GPU_NVIDIA'] = rec_req_df.apply(lambda row: fill_empty_with_other(row, 'GPU_NVIDIA', 'GPU_AMD'), axis=1)
rec_req_df['GPU_Intel'] = rec_req_df.apply(lambda row: fill_empty_with_other(row, 'GPU_Intel', 'GPU_NVIDIA'), axis=1)
rec_req_df['GPU_Intel'] = rec_req_df.apply(lambda row: fill_empty_with_other(row, 'GPU_Intel', 'GPU_AMD'), axis=1)

# Matching skor
rec_req_df['CPU_Intel_score'] = rec_req_df['CPU_Intel'].apply(match_cpu_score)
rec_req_df['CPU_AMD_score'] = rec_req_df['CPU_AMD'].apply(match_cpu_score)
rec_req_df['GPU_NVIDIA_score'] = rec_req_df['GPU_NVIDIA'].apply(match_gpu_score)
rec_req_df['GPU_AMD_score'] = rec_req_df['GPU_AMD'].apply(match_gpu_score)
rec_req_df['GPU_Intel_score'] = rec_req_df['GPU_Intel'].apply(match_gpu_score)

# Update GPU_NVIDIA_score & GPU_AMD_score kalau kosong, pakai GPU_Intel_score
rec_req_df.loc[rec_req_df['GPU_NVIDIA_score'] == 0, 'GPU_NVIDIA_score'] = rec_req_df['GPU_Intel_score']
rec_req_df.loc[rec_req_df['GPU_AMD_score'] == 0, 'GPU_AMD_score'] = rec_req_df['GPU_Intel_score']

# Cek hasil
print("\n=== Hasil Skor Minimum Requirements (FuzzyWuzzy) ===")
print(min_req_df[['Game', 'CPU_Intel', 'CPU_AMD', 'CPU_Intel_score', 'CPU_AMD_score',
                   'GPU_NVIDIA', 'GPU_AMD', 'GPU_Intel',
                   'GPU_NVIDIA_score', 'GPU_AMD_score', 'GPU_Intel_score']].head())

print("\n=== Hasil Skor Recommended Requirements (FuzzyWuzzy) ===")
print(rec_req_df[['Game', 'CPU_Intel', 'CPU_AMD', 'CPU_Intel_score', 'CPU_AMD_score',
                   'GPU_NVIDIA', 'GPU_AMD', 'GPU_Intel',
                   'GPU_NVIDIA_score', 'GPU_AMD_score', 'GPU_Intel_score']].head())

In [ ]:
# Simpan rec_req_df ke CSV
rec_req_df.to_csv('recommended_requirements_processed.csv', index=False)
print("DataFrame 'rec_req_df' telah disimpan ke 'recommended_requirements_processed.csv'")

# Simpan min_req_df ke CSV
min_req_df.to_csv('minimum_requirements_processed.csv', index=False)
print("DataFrame 'min_req_df' telah disimpan ke 'minimum_requirements_processed.csv'")

In [ ]:
from fuzzywuzzy import process
import pandas as pd

# Lookup dictionary
cpu_lookup = dict(zip(cpu_bench_df['CPU Model'], cpu_bench_df['CPU Score']))
gpu_lookup = dict(zip(gpu_bench_df['GPU Model'], gpu_bench_df['GPU Score']))

# Fungsi matching CPU
def match_cpu_score(cpu_name):
    if pd.isna(cpu_name) or cpu_name.strip() == '':
        return 0
    match = process.extractOne(cpu_name, cpu_lookup.keys(), score_cutoff=80)
    if match:
        return cpu_lookup[match[0]]
    return 0

# Fungsi matching GPU
def match_gpu_score(gpu_name):
    if pd.isna(gpu_name) or gpu_name.strip() == '':
        return 0
    if 'Integrated' in gpu_name or gpu_name.lower() == 'integrated':
        return 300
    match = process.extractOne(gpu_name, gpu_lookup.keys(), score_cutoff=80)
    if match:
        return gpu_lookup[match[0]]
    return 0

# Proses pencarian skor untuk laptop_df
laptop_df['CPU_score'] = laptop_df['CPU'].apply(match_cpu_score)
laptop_df['GPU_score'] = laptop_df['GPU'].apply(match_gpu_score)

# Hasil
print("\n=== Hasil Skor Dataset Laptop ===")
print(laptop_df[['Brand', 'Model', 'CPU', 'CPU_score', 'GPU', 'GPU_score', 'Final Price']].head())

In [ ]:
# Simpan laptop_df ke CSV
laptop_df.to_csv('laptop_data_processed.csv', index=False)
print("DataFrame 'laptop_df' telah disimpan ke 'laptop_data_processed.csv'")

In [ ]:
def clean_ram(ram_str):
    if pd.isna(ram_str) or ram_str.strip() == '' or ram_str.lower() == 'unknown':
        return 0  # default ke 0 jika tidak diketahui

    ram_str = str(ram_str).lower().strip()
    if 'gb' in ram_str:
        return int(float(ram_str.replace('gb', '').strip()))
    elif 'mb' in ram_str:
        mb_value = float(ram_str.replace('mb', '').strip())
        return int(mb_value / 1024) if mb_value >= 512 else 1
    else:
        # fallback jika tidak ada satuan, tapi string bisa dikonversi
        try:
            return int(float(ram_str))
        except ValueError:
            return 0  # fallback jika tidak bisa dikonversi

# Terapkan
min_req_df['RAM'] = min_req_df['RAM'].apply(clean_ram)
rec_req_df['RAM'] = rec_req_df['RAM'].apply(clean_ram)

In [ ]:
def clean_file_size(size_str):
    if pd.isna(size_str) or size_str.strip() == '' or size_str.lower() == 'unknown':
        return 0

    size_str = str(size_str).lower().strip()
    if 'gb' in size_str:
        return int(float(size_str.replace('gb', '').strip()))
    elif 'mb' in size_str:
        mb_value = float(size_str.replace('mb', '').strip())
        return int(mb_value / 1024) if mb_value >= 512 else 1
    else:
        # fallback jika tidak ada satuan, tapi string bisa dikonversi
        try:
            return int(float(size_str))
        except ValueError:
            return 0

# Terapkan
min_req_df['File Size'] = min_req_df['File Size'].apply(clean_file_size)
rec_req_df['File Size'] = rec_req_df['File Size'].apply(clean_file_size)

In [ ]:
def categorize_laptops(game_name, df_min, df_rec, df_laptops, ram_weight=1.0, storage_weight=1.0):
    game_req = df_min[df_min['Game'].str.lower() == game_name.lower()]
    game_rec = df_rec[df_rec['Game'].str.lower() == game_name.lower()]
    if game_req.empty or game_rec.empty:
        print(f"Game '{game_name}' tidak ditemukan.")
        return pd.DataFrame()

    req_row_min = game_req.iloc[0]
    req_row_rec = game_rec.iloc[0]

    ram_req = req_row_min['RAM']
    storage_req = req_row_min['File Size']

    # --- Vendor detection helper ---
    def get_gpu_vendor(gpu_name):
        gpu_name = gpu_name.lower()
        if any(x in gpu_name for x in ['rtx', 'gtx', 'mx']):
            return 'nvidia'
        elif any(x in gpu_name for x in ['radeon', 'rx', 'vega', 'pro']):
            return 'amd'
        elif any(x in gpu_name for x in ['integrated', 'iris', 'uhd']):
            return 'intel'
        else:
            return 'intel'  # fallback

    # --- CPU detection helper ---
    def get_cpu_vendor(cpu_name):
        cpu_name = cpu_name.lower()
        if 'intel' in cpu_name:
            return 'intel'
        elif 'amd' in cpu_name:
            return 'amd'
        else:
            return 'intel'  # fallback

    # --- Ambil score ---
    def get_cpu_score(row, req_row):
        vendor = get_cpu_vendor(row['CPU'])
        if vendor == 'intel':
            return req_row['CPU_Intel_score']
        elif vendor == 'amd':
            return req_row['CPU_AMD_score']
        else:
            return req_row['CPU_Intel_score']

    def get_gpu_score(row, req_row):
        vendor = get_gpu_vendor(row['GPU'])
        if vendor == 'nvidia':
            return req_row['GPU_NVIDIA_score']
        elif vendor == 'amd':
            return req_row['GPU_AMD_score']
        elif vendor == 'intel':
            return req_row['GPU_Intel_score']
        else:
            return req_row['GPU_Intel_score']

    # --- Hitung match score ---
    def calculate_match(row):
        cpu_req = get_cpu_score(row, req_row_min)
        gpu_req = get_gpu_score(row, req_row_min)
        cpu_score = row['CPU_score'] / cpu_req if cpu_req > 0 else 1.0
        gpu_score = row['GPU_score'] / gpu_req if gpu_req > 0 else 1.0
        ram_score = (row['RAM'] / ram_req) * ram_weight if ram_req > 0 else 1.0
        storage_score = (row['Storage'] / storage_req) * storage_weight if storage_req > 0 else 1.0
        final_score = (cpu_score + gpu_score + ram_score + storage_score) / (2 + ram_weight + storage_weight)
        return final_score

    # --- Kategorisasi ---
    def categorize(row):
        cpu_min = get_cpu_score(row, req_row_min)
        gpu_min = get_gpu_score(row, req_row_min)
        cpu_rec = get_cpu_score(row, req_row_rec)
        gpu_rec = get_gpu_score(row, req_row_rec)

        if row['CPU_score'] < cpu_min or row['GPU_score'] < gpu_min:
            return 'Disqualified'

        cpu_rec_flag = row['CPU_score'] >= cpu_rec
        gpu_rec_flag = row['GPU_score'] >= gpu_rec

        if cpu_rec_flag and gpu_rec_flag:
            return 'Recommended'
        elif cpu_rec_flag or gpu_rec_flag:
            return 'Mixed'
        else:
            return 'Minimum'

    # --- Filter RAM & Storage ---
    df_filtered = df_laptops[
        (df_laptops['RAM'] >= ram_req) & (df_laptops['Storage'] >= storage_req)
    ].copy()

    # --- Hitung Match_Score ---
    df_filtered['Match_Score'] = df_filtered.apply(calculate_match, axis=1)

    # --- Tentukan Kategori ---
    df_filtered['Category'] = df_filtered.apply(categorize, axis=1)

    # --- Buang yang Disqualified ---
    df_final = df_filtered[df_filtered['Category'] != 'Disqualified'].copy()

    # --- Urutkan ---
    df_final = df_final.sort_values(by='Match_Score', ascending=False).reset_index(drop=True)
    return df_final

# Contoh pemanggilan
final_laptops = categorize_laptops('Genshin Impact', min_req_df, rec_req_df, laptop_df)
print(final_laptops[['Brand', 'Model', 'CPU', 'GPU', 'RAM', 'Storage', 'Category', 'Match_Score']])

In [ ]:
# Simpan DataFrame final_laptops ke CSV
final_laptops.to_csv('final_laptops_categorized.csv', index=False)
print("DataFrame 'final_laptops' telah disimpan ke 'final_laptops_categorized.csv'")

In [ ]:
import pandas as pd
import re

# --- 2. Cleaning kolom 'Description' ---
def clean_description(desc):
    """
    Bersihin newline, tanda kutip, spasi dobel, dan karakter aneh
    """
    if pd.isnull(desc):
        return ""
    # newline -> spasi
    desc = re.sub(r'\n', ' ', desc)
    # hapus tanda kutip
    desc = re.sub(r'["\']', '', desc)
    # hapus karakter aneh (non-ASCII)
    desc = re.sub(r'[^\x00-\x7F]+', ' ', desc)
    # hapus spasi dobel
    desc = re.sub(r'\s+', ' ', desc)
    return desc.strip()

rec_req_df['Description'] = rec_req_df['Description'].apply(clean_description)
min_req_df['Description'] = min_req_df['Description'].apply(clean_description)

# --- 4. Cek hasil cleaning ---
print(rec_req_df.head())
print(min_req_df.head())

# --- 5. Simpan dataset hasil cleaning (opsional) ---
rec_req_df.to_csv('recommended_clean.csv', index=False)
min_req_df.to_csv('minimum_clean.csv', index=False)

print("✅ Preprocessing selesai. Dataset lebih rapi!")

In [ ]:
# Knowledge Base 1: mapping game → minimum/recommended requirement

# Buat dictionary mapping game ke requirement
min_req_kb = min_req_df.set_index('Game').to_dict('index')
rec_req_kb = rec_req_df.set_index('Game').to_dict('index')

# Contoh akses
game_name = 'Assassin\'s Creed Shadows'
print("Minimum Requirement:", min_req_kb.get(game_name, {}))
print("Recommended Requirement:", rec_req_kb.get(game_name, {}))

In [ ]:
# Knowledge Base 2: mapping laptop → CPU, GPU, RAM score, and Brand

from collections import defaultdict

# Create a dictionary mapping laptop Model to a list of specs (including Brand)
# Using defaultdict(list) allows easily appending specs for multiple configurations
# of the same model.
laptop_kb = defaultdict(list)

for _, row in laptop_df.iterrows():
    model = row['Model']
    specs = {
        'Brand': row['Brand'], # Add Brand here
        'CPU_score': row['CPU_score'],
        'GPU_score': row['GPU_score'],
        'RAM': row['RAM'],
        'Final Price': row['Final Price'], # Add Final Price as well, it might be useful
        'Storage': row['Storage'], # Add Storage
        'Storage type': row['Storage type'] # Add Storage type
    }
    laptop_kb[model].append(specs)

# Contoh akses
laptop_name = 'ROG'
print("Laptop Specs for", laptop_name, ":", laptop_kb.get(laptop_name, {}))

# Contoh akses untuk melihat brand
if laptop_kb.get(laptop_name):
    print("Example Brand for", laptop_name, ":", laptop_kb[laptop_name][0].get('Brand'))

# Example access for a different model
laptop_name_2 = 'Katana' # Replace with another model from your data
print("\nLaptop Specs for", laptop_name_2, ":", laptop_kb.get(laptop_name_2, {}))

In [ ]:
category_kb = {}

# Loop semua game dan ambil categorynya
for idx, row in min_req_df.iterrows():
    categories = row['Category']
    if isinstance(categories, list):  # sudah list
        for cat in categories:
            cat_lower = cat.lower().strip()
            if cat_lower not in category_kb:
                category_kb[cat_lower] = set()
            category_kb[cat_lower].add(row['Game'])

# Contoh akses
print("Games in 'racing':", category_kb.get('racing', set()))
print("Games in 'racing':", category_kb.get('balapan', set()))

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Ambil deskripsi game
descriptions = min_req_df['Description'].fillna('').tolist()

# Inisialisasi model multilingual
# 'paraphrase-multilingual-MiniLM-L12-v2' adalah model multilingual yang cukup baik
# yang dilatih di banyak pasangan bahasa.
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Buat embeddings
description_embeddings = model.encode(descriptions, convert_to_tensor=True)

print("Shape embeddings:", description_embeddings.shape)

In [ ]:
# Re-run the search_games cell to test with the new multilingual model
import torch

def search_games(query, model, description_embeddings, df, top_n=5, category_kb=None, category_boost=0.1):
    # Encode query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Hitung cosine similarity awal
    similarities = torch.nn.functional.cosine_similarity(query_embedding, description_embeddings)

    # Preprocess query untuk category matching
    query_tokens_cleaned = basic_preprocessing(query)
    query_tokens_cleaned = remove_stopwords(query_tokens_cleaned)
    query_tokens_cleaned = map_synonyms(query_tokens_cleaned)

    # Beri bobot berdasarkan kategori yang cocok
    boosted_similarities = similarities.clone() # Salin skor kesamaan awal
    for i in range(len(df)):
        game_categories = df.iloc[i]["Category"]
        if isinstance(game_categories, list):
            # Check if any cleaned query token matches any category of the game
            # Using a set for faster lookups
            game_categories_lower = set([cat.lower() for cat in game_categories])
            if any(token in game_categories_lower for token in query_tokens_cleaned):
                boosted_similarities[i] += category_boost # Tambahkan bobot jika ada kecocokan kategori

    # Urutkan berdasarkan boosted similarities
    top_indices = boosted_similarities.argsort(descending=True)[:top_n]

    results = []
    for idx in top_indices:
        idx = idx.item()  # Convert tensor to int!
        results.append({
            "game": df.iloc[idx]["Game"],
            "category": df.iloc[idx]["Category"],
            "similarity": boosted_similarities[idx].item() # Gunakan skor yang sudah diberi bobot
        })

    return results

# Contoh penggunaan dengan bobot kategori
query = "game balapan"
results = search_games(query, model, description_embeddings, min_req_df, top_n=10, category_boost=0.4) # Naikan bobot sedikit

print("Hasil pencarian (dengan bobot kategori):")
for res in results:
    print(res)

query_2 = "laptop buat game petualangan dunia terbuka"
results_2 = search_games(query_2, model, description_embeddings, min_req_df, top_n=10, category_boost=0.2)

print("\nHasil pencarian untuk 'laptop buat game petualangan dunia terbuka' (dengan bobot kategori):")
for res in results_2:
    print(res)

In [ ]:
import re
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import RegexpTokenizer
from fuzzywuzzy import process # Import process
from collections import defaultdict # Import defaultdict

# ✅ Gunakan tokenizer berbasis regex untuk menghindari dependency resource spesifik
tokenizer = RegexpTokenizer(r'\w+')

# Inisialisasi stopword remover Sastrawi
factory = StopWordRemoverFactory()
stopwords_sastrawi = set(factory.get_stop_words())

# Tambahkan beberapa stopwords umum yang mungkin muncul di query rekomendasi
additional_stopwords = {'cocok', 'buat', 'main', 'dengan', 'rekomendasi', 'spesifikasi', 'spek', 'apa', 'yang', 'game'}
stopwords_sastrawi.update(additional_stopwords)

# === 2️⃣ Stopword Removal ===
def remove_stopwords(tokens):
    return [t for t in tokens if t not in stopwords_sastrawi]

# === 1️⃣ Basic Preprocessing ===
def basic_preprocessing(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}’‘]", " ", text)  # tambah karakter khusus
    tokens = tokenizer.tokenize(text)  # tokenizer regex
    return tokens

# === Create Keyword to Game Mapping (with uniqueness check) ===
def create_keyword_game_map(game_list, stopwords):
    keyword_candidate_map = defaultdict(list)
    # First pass: collect all games for each keyword
    for game in game_list:
        tokens = basic_preprocessing(game)
        tokens = remove_stopwords(tokens)
        for token in tokens:
            if len(token) > 2: # Only consider keywords longer than 2 characters
                keyword_candidate_map[token].append(game)

    # Second pass: identify unique keywords
    unique_keyword_map = {}
    for keyword, games in keyword_candidate_map.items():
        if len(games) == 1: # If the keyword is associated with only one game
            unique_keyword_map[keyword] = games[0] # Map keyword to the single game title

    return unique_keyword_map # Return map of unique keywords to single game titles


game_list = min_req_df['Game'].tolist()
# Create the keyword map for games
unique_keyword_game_map = create_keyword_game_map(game_list, stopwords_sastrawi)

# === 3️⃣ Named Entity Recognition & Budget Extraction ===
def extract_entities_and_budget(user_query, game_list, laptop_list, laptop_brand_list, unique_keyword_game_map, score_cutoff_high=95, score_cutoff_low_game=70, score_cutoff_low_laptop=80):
    found_games = set()
    found_laptops = set() # This will now include both models and brands
    extracted_budget = None

    query_lower = user_query.lower()
    query_tokens = basic_preprocessing(user_query) # Tokenize query
    query_tokens_cleaned = remove_stopwords(query_tokens) # Remove stopwords from query

    # Reconstruct the cleaned query string for fuzzy matching
    cleaned_query_string = " ".join(query_tokens_cleaned)

    # --- Extract Budget ---
    # Look for patterns like "10 juta", "15jt", "500 ribu", "rp 20000000", or standalone large numbers
    # Updated regex to also look for numbers with at least 6 digits, possibly preceded by 'budget' or 'harga'
    budget_match = re.search(r'(\d+)\s*(juta|jt|ribu|rb|k)|rp\s*(\d+)|(?:budget|harga)\s*(\d{6,})|(\d{6,})', query_lower)

    if budget_match:
        # Prioritize patterns with units or 'rp'
        if budget_match.group(1): # Patterns with units
            value_str = budget_match.group(1)
            unit = budget_match.group(2)
            try:
                value = int(value_str)
                if unit in ['juta', 'jt']:
                    extracted_budget = value * 1_000_000
                elif unit in ['ribu', 'rb', 'k']:
                     extracted_budget = value * 1_000
            except ValueError:
                extracted_budget = None # Invalid number format
        elif budget_match.group(3): # 'rp 12345' pattern
            value_str = budget_match.group(3)
            try:
                extracted_budget = int(value_str)
            except ValueError:
                extracted_budget = None # Invalid number format
        elif budget_match.group(4): # (?:budget|harga)\s*(\d{6,}) pattern
             value_str = budget_match.group(4)
             try:
                 extracted_budget = int(value_str)
             except ValueError:
                 extracted_budget = None # Invalid number format
        elif budget_match.group(5): # (\d{6,}) standalone large number
            value_str = budget_match.group(5)
            try:
                extracted_budget = int(value_str)
            except ValueError:
                extracted_budget = None # Invalid number format


    # --- 3a: Unique Keyword Matching for Games (Prioritized) ---
    unique_game_found = False
    for token in query_tokens_cleaned:
        if token in unique_keyword_game_map:
            found_games.add(unique_keyword_game_map[token])
            unique_game_found = True
            break # Prioritize the first unique keyword found and stop game search

    # --- 3b: Fuzzy Matching for Games (Only if no unique keyword found) ---
    if not unique_game_found:
        matches_games = process.extract(cleaned_query_string, game_list, limit=15)
        for match, score in matches_games:
            if score >= score_cutoff_low_game and match[0] in game_list and len(match[0]) > 1:
                 if re.search(re.escape(match[0].lower()), cleaned_query_string):
                      found_games.add(match[0])

    # --- 3c: Token Containment Fallback for Games (If no games found yet) ---
    if not found_games:
        for token in query_tokens_cleaned:
            if len(token) > 2:
                 for game in game_list:
                      if re.search(r'\b' + re.escape(token) + r'\b', game.lower()):
                           found_games.add(game)

    # --- 3d: Exact/High-Confidence Fuzzy Matching for Laptops (Models and Brands) ---
    # Combine laptop models and brands for matching
    laptop_entities = set(laptop_list + laptop_brand_list)

    for entity in laptop_entities:
         # Check for exact match first
         if entity.lower() in cleaned_query_string:
              found_laptops.add(entity)
              continue # Move to next entity if exact match found

         match = process.extractOne(entity, [cleaned_query_string], score_cutoff=score_cutoff_high)
         if match:
             found_laptops.add(match[0])

    # --- 3e: Corrected Lower Cutoff Fuzzy Matching for Laptops (Fallback if no high-confidence found) ---
    if not found_laptops:
        # Iterate through combined laptop entities and match each against the cleaned query string
        for entity in laptop_entities:
            match = process.extractOne(entity, [cleaned_query_string], score_cutoff=score_cutoff_low_laptop)
            if match:
                # Check if the matched entity name (from the list) is present in the cleaned query string
                if re.search(re.escape(entity.lower()), cleaned_query_string):
                    found_laptops.add(entity)

    return list(found_games), list(found_laptops), extracted_budget

# === 4️⃣ Synonym Mapping ===
SYNONYM_MAPPING = {
    # Action
    "aksi": "action",
    "laga": "action",
    "action": "action",

    # Adventure
    "petualangan": "adventure",
    "openworld": "adventure",
    "open-world": "adventure",
    "adventure": "adventure",

    # Animation & Modeling
    "animasi": "animation & modeling",
    "modeling": "animation & modeling",
    "animasi 3d": "animation & modeling",

    # Casual
    "santai": "casual",
    "kasual": "casual",
    "casual": "casual",

    # Design & Illustration
    "desain": "design & illustration",
    "ilustrasi": "design & illustration",
    "gambar": "design & illustration",

    # Early Access
    "akses awal": "early access",
    "pre-release": "early access",

    # FPS
    "tembak": "fps",
    "tembak-tembakan": "fps",
    "first person": "fps",
    "fps": "fps",

    # Fighting
    "pertarungan": "fighting",
    "fighter": "fighting",
    "bela diri": "fighting",

    # Free to Play
    "gratis": "free to play",
    "f2p": "free to play",
    "free": "free to play",

    # Game Development
    "pengembangan game": "game development",
    "dev game": "game development",
    "pembuatan game": "game development",

    # Gore
    "kekerasan grafis": "gore",
    "darah": "gore",
    "gore": "gore",

    # Indie
    "indie": "indie",
    "independen": "indie",
    "permainan indie": "indie",

    # JRPG
    "rpg jepang": "jrpg",
    "jrpg": "jrpg",
    "anime rpg": "jrpg",

    # MMO
    "mmo": "mmo",
    "multiplayer masif": "mmo",
    "game online": "mmo",

    # MOBA
    "moba": "moba",
    "arena battle": "moba",
    "dota-like": "moba",

    # Photo Editing
    "edit foto": "photo editing",
    "foto": "photo editing",
    "photoshop": "photo editing",

    # Platformer
    "platform": "platformer",
    "lompat": "platformer",
    "arcade": "platformer",

    # RPG
    "role playing": "rpg",
    "rpg": "rpg",
    "peran": "rpg",

    # Racing
    "balapan": "racing",
    "balap": "racing",
    "racing": "racing",

    # Rogue-like
    "roguelike": "rogue-like",
    "permadeath": "rogue-like",
    "prosedural": "rogue-like",

    # Sexual Content
    "konten dewasa": "sexual content",
    "seksual": "sexual content",
    "dewasa": "sexual content",

    # Simulation
    "simulasi": "simulation",
    "sim": "simulation",
    "simulator": "simulation",

    # Space
    "angkasa": "space",
    "luar angkasa": "space",
    "space": "space",

    # Sports
    "olahraga": "sports",
    "sport": "sports",
    "olahraga elektronik": "sports",

    # Strategy
    "strategi": "strategy",
    "strategic": "strategy",
    "taktik": "strategy",

    # Third-person shooter
    "tps": "third-person shooter",
    "shooter orang ketiga": "third-person shooter",
    "third person": "third-person shooter",

    # Turn-based
    "giliran": "turn-based",
    "turn based": "turn-based",
    "tb": "turn-based",

    # Unknown
    "tidak diketahui": "unknown",
    "unknown": "unknown",
    "misteri": "unknown",

    # Utilities
    "alat": "utilities",
    "tool": "utilities",
    "utilitas": "utilities",

    # Video Production
    "produksi video": "video production",
    "editing video": "video production",
    "video editor": "video production",

    # Violent
    "kekerasan": "violent",
    "violent": "violent",
    "brutal": "violent"
}

def map_synonyms(tokens):
    return [SYNONYM_MAPPING.get(t, t) for t in tokens]

# === Update Pipeline Function ===
def nlp_pipeline_fuzzy(user_query, game_list, laptop_list, laptop_brand_list, unique_keyword_game_map):
    tokens = basic_preprocessing(user_query)
    tokens = remove_stopwords(tokens)
    tokens = map_synonyms(tokens)
    # Use the improved entity extraction and budget extraction function, passing brand list
    found_games, found_laptops, extracted_budget = extract_entities_and_budget(user_query, game_list, laptop_list, laptop_brand_list, unique_keyword_game_map)

    return {
        "tokens": tokens,
        "found_games": found_games,
        "found_laptops": found_laptops, # This now includes both models and brands
        "budget": extracted_budget
    }

# === Contoh penggunaan dengan fuzzy matching ===
game_list = min_req_df['Game'].tolist()
laptop_list = laptop_df['Model'].tolist()
# Get the list of unique laptop brands
laptop_brand_list = laptop_df['Brand'].unique().tolist()

unique_keyword_game_map = create_keyword_game_map(game_list, stopwords_sastrawi) # Create the unique keyword map

# Test cases
queries = [
    "Laptop yang cocok buat main assasin creed apa dengan laptop ASUS?",
    "Laptop yang cocok buat main Genshin apa dengan laptop Katana?",
    "Laptop yang cocok buat main Honkai apa dengan laptop Voom?",
    "Rekomendasi Laptop yang cocok buat main Honkai apa dengan laptop V15?",
    "Spek laptop TUF buat game Valorant",
    "Laptop yang cocok buat main assasin creed odyssey apa dengan laptop ROG?",
    "Laptop msi buat game assassin",
    "Rekomendasi laptop Omen untuk Genshin Impact dengan budget 20 juta",
    "Cari laptop untuk game balapan dengan budget 15jt",
    "Saya butuh laptop untuk main Game Pass dibawah 10 ribu rupiah",
    "Laptop gaming seharga 500 ribu",
    "Budget saya 30000000 untuk laptop gaming",
    "Laptop Asus untuk game Valorant",
    "Cari laptop Acer buat game balapan",
    "Butuh laptop HP budget 10 juta"
]

for query in queries:
    # Pass the brand list to the pipeline
    result = nlp_pipeline_fuzzy(query, game_list, laptop_list, laptop_brand_list, unique_keyword_game_map)
    print(f"\nQuery: {query}\nResult: {result}")

In [ ]:
import pandas as pd
import torch # Import torch for semantic search
from fuzzywuzzy import process # Import process for fuzzy matching

# --- Fungsi Rekomendasi Utama ---
def get_laptop_recommendations(user_query, laptop_df, min_req_df, rec_req_df,
                               min_req_kb, rec_req_kb,
                               model, description_embeddings,
                               laptop_list, laptop_brand_list, unique_keyword_game_map,
                               category_boost_value=0.2, # Bobot kategori untuk semantic search
                               top_n_semantic_games=5 # Jumlah game teratas dari semantic search jika tidak ada game spesifik
                               ):

    print(f"Query Pengguna: {user_query}")

    # 1. Jalankan NLP Pipeline
    pipeline_result = nlp_pipeline_fuzzy(user_query, min_req_df['Game'].tolist(), laptop_df['Model'].tolist(), laptop_df['Brand'].unique().tolist(), unique_keyword_game_map)
    found_games = pipeline_result['found_games']
    found_laptops_entities = pipeline_result['found_laptops'] # Bisa berisi brand atau model
    extracted_budget = pipeline_result['budget']

    print(f"Hasil NLP Pipeline: Game Terdeteksi={found_games}, Laptop Terdeteksi={found_laptops_entities}, Budget Terdeteksi={extracted_budget}")

    # --- Tambahkan logika pengecekan game terdeteksi ---
    if not found_games:
        print("Tidak ada game spesifik terdeteksi dari query.")
        # Cek apakah hanya ada budget dan/atau laptop spesifik
        if found_laptops_entities or extracted_budget is not None and extracted_budget > 0:
             # Jika ada info laptop/budget tapi tidak ada game, tanya pengguna
             return pd.DataFrame({"Status": ["Butuh Info Game"], "Pesan": ["Mohon maaf, saya belum mendeteksi game spesifik dari query Anda. Apakah ada game tertentu yang ingin Anda mainkan di laptop baru Anda?"]})
        else:
             # Jika tidak ada info game, laptop, atau budget, query terlalu umum
             print("Query terlalu umum.")
             # Fallback ke semantic search untuk game jika query tidak terlalu pendek dan relevan
             query_tokens_cleaned = basic_preprocessing(user_query)
             query_tokens_cleaned = remove_stopwords(query_tokens_cleaned)
             query_tokens_cleaned = map_synonyms(query_tokens_cleaned)

             if len(query_tokens_cleaned) > 1: # Jika masih ada token relevan setelah preprocessing
                  print("Melakukan Semantic Search untuk game berdasarkan query umum...")
                  semantic_results = search_games(user_query, model, description_embeddings, min_req_df, top_n=top_n_semantic_games, category_boost=category_boost_value)
                  game_targets = [res['game'] for res in semantic_results if res['similarity'] > 0.1] # Ambil game dengan similarity > threshold
                  if game_targets:
                       print(f"Menemukan game relevan dari Semantic Search: {game_targets}")

                  else:
                       print("Semantic Search tidak menemukan game relevan.")
                       return pd.DataFrame({"Status": ["Tidak Ditemukan"], "Pesan": ["Tidak dapat menemukan game relevan dari query Anda."]})
             else:
                  print("Query terlalu pendek atau tidak relevan setelah preprocessing.")
                  return pd.DataFrame({"Status": ["Tidak Ditemukan"], "Pesan": ["Query Anda terlalu umum atau tidak jelas. Mohon berikan nama game, brand/model laptop, atau budget yang spesifik."]})

    if found_games:
        game_targets = found_games
        print(f"Menggunakan game yang terdeteksi spesifik: {game_targets}")

    target_game_min_req = None
    target_game_rec_req = None

    if game_targets: # Pastikan game_targets tidak kosong setelah semua pengecekan/semantic search
        first_game = game_targets[0] # Ambil game pertama sebagai target utama
        target_game_min_req = min_req_kb.get(first_game)
        target_game_rec_req = rec_req_kb.get(first_game)

        if not target_game_min_req or not target_game_rec_req:
            print(f"Persyaratan game '{first_game}' tidak ditemukan di knowledge base.")
            # Fallback: coba game target berikutnya jika ada
            for next_game in game_targets[1:]:
                 target_game_min_req = min_req_kb.get(next_game)
                 target_game_rec_req = rec_req_kb.get(next_game)
                 if target_game_min_req and target_game_rec_req:
                      first_game = next_game # Update game target utama
                      print(f"Menggunakan persyaratan game alternatif: {first_game}")
                      break # Gunakan game pertama yang persyaratannya ditemukan

            if not target_game_min_req or not target_game_rec_req:
                 print("Tidak dapat menemukan persyaratan yang valid untuk game target.")
                 return pd.DataFrame({"Status": ["Tidak Ditemukan"], "Pesan": [f"Persyaratan untuk game '{first_game}' atau game relevan lainnya tidak ditemukan."]})

        print(f"Persyaratan Min: {target_game_min_req['CPU_Intel']} / {target_game_min_req['GPU_NVIDIA']}, RAM: {target_game_min_req['RAM']}, Storage: {target_game_min_req['File Size']}")
        print(f"Persyaratan Rec: {target_game_rec_req['CPU_Intel']} / {target_game_rec_req['GPU_NVIDIA']}, RAM: {target_game_rec_req['RAM']}, Storage: {target_game_rec_req['File Size']}")
    else:
         # Ini seharusnya sudah ditangani di blok if not found_games di atas,
         # tapi sebagai fallback keamanan:
         print("Tidak ada game relevan yang ditemukan dari query setelah semua upaya.")
         return pd.DataFrame({"Status": ["Tidak Ditemukan"], "Pesan": ["Tidak dapat menemukan game relevan dari query Anda."]})


    # 3. Siapkan Daftar Laptop untuk Dievaluasi (Filter Budget, Brand, Model)
    laptops_to_evaluate = laptop_df.copy()

    # Filter Budget
    if extracted_budget is not None and extracted_budget > 0:
        print(f"Memfilter laptop dengan budget <= {extracted_budget}")
        laptops_to_evaluate = laptops_to_evaluate[laptops_to_evaluate['Final Price'] <= extracted_budget].copy()
        print(f"Jumlah laptop setelah filter budget: {len(laptops_to_evaluate)}") # Debug print: Jumlah laptop setelah filter budget
        if laptops_to_evaluate.empty:
            print("Tidak ada laptop dalam budget yang terdeteksi.")
            return pd.DataFrame({"Status": ["Tidak Ditemukan"], "Pesan": [f"Tidak ada laptop dalam budget {extracted_budget:,} yang tersedia."]})

    # Filter Brand atau Model Spesifik
    if found_laptops_entities:
        print(f"Memfilter laptop berdasarkan entitas: {found_laptops_entities}")
        filter_mask = pd.Series([False] * len(laptops_to_evaluate), index=laptops_to_evaluate.index)
        for entity in found_laptops_entities:
            if entity in laptop_brand_list:
                filter_mask |= (laptops_to_evaluate['Brand'].str.lower() == entity.lower())
            elif entity in laptop_list:
                 filter_mask |= (laptops_to_evaluate['Model'].str.lower() == entity.lower())
            else:
                 brand_match = process.extractOne(entity, laptops_to_evaluate['Brand'].tolist(), score_cutoff=90)
                 model_match = process.extractOne(entity, laptops_to_evaluate['Model'].tolist(), score_cutoff=90)
                 if brand_match and brand_match[1] >= 90:
                      filter_mask |= (laptops_to_evaluate['Brand'].str.lower() == brand_match[0].lower())
                 if model_match and model_match[1] >= 90:
                      filter_mask |= (laptops_to_evaluate['Model'].str.lower() == model_match[0].lower())

        laptops_to_evaluate = laptops_to_evaluate[filter_mask].copy()
        print(f"Jumlah laptop setelah filter entitas: {len(laptops_to_evaluate)}") # Debug print: Jumlah laptop setelah filter entitas

        if laptops_to_evaluate.empty:
            print(f"Tidak ada laptop yang cocok dengan entitas '{', '.join(found_laptops_entities)}' dalam budget yang terdeteksi.")
            return pd.DataFrame({"Status": ["Tidak Ditemukan"], "Pesan": [f"Tidak ada laptop yang cocok dengan '{', '.join(found_laptops_entities)}' dalam kriteria Anda."]})


    # 4. Kategorisasi Laptop (Menggunakan categorize_laptops)
    def categorize_laptops_adapted(df_laptops_filtered, req_row_min, req_row_rec, ram_weight=1.0, storage_weight=1.0):
        if df_laptops_filtered.empty:
             return pd.DataFrame()

        ram_req = req_row_min['RAM']
        storage_req = req_row_min['File Size']

        def get_gpu_vendor(gpu_name):
            if pd.isna(gpu_name): return 'intel'
            gpu_name = str(gpu_name).lower()
            if any(x in gpu_name for x in ['rtx', 'gtx', 'mx']): return 'nvidia'
            elif any(x in gpu_name for x in ['radeon', 'rx', 'vega', 'pro']): return 'amd'
            elif any(x in gpu_name for x in ['integrated', 'iris', 'uhd', 'hd graphics']): return 'intel'
            else: return 'intel'

        def get_cpu_vendor(cpu_name):
            if pd.isna(cpu_name): return 'intel'
            cpu_name = str(cpu_name).lower()
            if 'intel' in cpu_name or any(x in cpu_name for x in ['core', 'pentium', 'celeron', 'xeon', 'evo']): return 'intel'
            elif 'amd' in cpu_name or any(x in cpu_name for x in ['ryzen', 'fx', 'athlon', 'phenom', 'opteron']): return 'amd'
            else: return 'intel'

        def get_cpu_req_score(row, req_row):
            vendor = get_cpu_vendor(row['CPU'])
            if vendor == 'intel': return req_row['CPU_Intel_score']
            elif vendor == 'amd': return req_row['CPU_AMD_score']
            else: return req_row['CPU_Intel_score']

        def get_gpu_req_score(row, req_row):
            vendor = get_gpu_vendor(row['GPU'])
            if vendor == 'nvidia': return req_row['GPU_NVIDIA_score']
            elif vendor == 'amd': return req_row['GPU_AMD_score']
            elif vendor == 'intel': return req_row['GPU_Intel_score']
            else: return req_row['GPU_Intel_score']

        def calculate_match(row):
            cpu_req = get_cpu_req_score(row, req_row_rec)
            gpu_req = get_gpu_req_score(row, req_row_rec)

            cpu_score_ratio = row['CPU_score'] / cpu_req if cpu_req > 0 else 1.0
            gpu_score_ratio = row['GPU_score'] / gpu_req if gpu_req > 0 else 1.0
            ram_score_ratio = (row['RAM'] / ram_req) * ram_weight if ram_req > 0 else 1.0
            storage_score_ratio = (row['Storage'] / storage_req) * storage_weight if storage_req > 0 else 1.0

            cpu_score_ratio = min(cpu_score_ratio, 5.0)
            gpu_score_ratio = min(gpu_score_ratio, 5.0)
            ram_score_ratio = min(ram_score_ratio, 5.0)
            storage_score_ratio = min(storage_score_ratio, 5.0)

            final_score = (cpu_score_ratio + gpu_score_ratio + ram_score_ratio + storage_score_ratio) / (2 + ram_weight + storage_weight)
            return final_score

        def categorize(row):
            cpu_min = get_cpu_req_score(row, req_row_min)
            gpu_min = get_gpu_req_score(row, req_row_min)
            cpu_rec = get_cpu_req_score(row, req_row_rec)
            gpu_rec = get_gpu_req_score(row, req_row_rec)

            # Check minimum requirements first
            if row['CPU_score'] < cpu_min or row['GPU_score'] < gpu_min or row['RAM'] < ram_req or row['Storage'] < storage_req:
                 return 'Disqualified'

            cpu_rec_flag = row['CPU_score'] >= cpu_rec
            gpu_rec_flag = row['GPU_score'] >= gpu_rec

            if cpu_rec_flag and gpu_rec_flag: return 'Recommended'
            elif cpu_rec_flag or gpu_rec_flag: return 'Mixed'
            else: return 'Minimum'

        df_filtered_req = df_laptops_filtered[
            (df_laptops_filtered['RAM'] >= ram_req) & (df_laptops_filtered['Storage'] >= storage_req)
        ].copy()
        print(f"Jumlah laptop setelah filter RAM/Storage minimum: {len(df_filtered_req)}") # Debug print: Jumlah laptop setelah filter RAM/Storage

        if df_filtered_req.empty:
             print("Tidak ada laptop yang memenuhi persyaratan RAM dan Storage minimum.")
             return pd.DataFrame()

        df_filtered_req['Match_Score'] = df_filtered_req.apply(calculate_match, axis=1)
        df_filtered_req['Category'] = df_filtered_req.apply(categorize, axis=1)

        df_final = df_filtered_req[df_filtered_req['Category'] != 'Disqualified'].copy()
        df_final = df_final.sort_values(by='Match_Score', ascending=False).reset_index(drop=True)
        return df_final[['Brand', 'Model', 'CPU', 'GPU', 'RAM', 'Storage', 'Final Price', 'Category', 'Match_Score']]


    print("Mengkategorikan laptop...")
    final_recommendations = categorize_laptops_adapted(
        laptops_to_evaluate,
        target_game_min_req,
        target_game_rec_req
    )

    # 5. Tampilkan Hasil
    if final_recommendations.empty:
        print("\nTidak ada rekomendasi laptop yang ditemukan berdasarkan kriteria Anda.")
        print("Tidak ada laptop yang memenuhi persyaratan minimum game yang relevan di antara laptop yang sudah difilter.")
        return pd.DataFrame({"Status": ["Tidak Ditemukan"], "Pesan": ["Tidak ada rekomendasi laptop yang memenuhi persyaratan minimum game yang relevan dalam kriteria Anda."]})
    else:
        print("\nHasil Rekomendasi Laptop:")
        final_recommendations['Category_Order'] = final_recommendations['Category'].apply(
            lambda x: 0 if x == 'Recommended' else (1 if x == 'Mixed' else 2)
        )
        final_recommendations = final_recommendations.sort_values(by=['Category_Order', 'Match_Score'], ascending=[True, False]).drop(columns='Category_Order')
        display(final_recommendations)
        return final_recommendations


# --- Jalankan ulang query yang bermasalah dengan debug prints ---
query_problem = "Laptop yang cocok buat main valorant dengan laptop Asus dengan harga 100 juta"
recommendations_debug = get_laptop_recommendations(
    query_problem, laptop_df, min_req_df, rec_req_df,
    min_req_kb, rec_req_kb,
    model, description_embeddings,
    laptop_df['Model'].tolist(), laptop_df['Brand'].unique().tolist(), unique_keyword_game_map
)